# Basic LM Studio Interaction

This example demonstrates how to load a model and generate a basic response to a question.

In [1]:
import lmstudio as lms

model = lms.llm("qwen3-0.6b")
result = model.respond("What is the meaning of life?")

print(result)

<think>
Okay, so the user asked, "What is the meaning of life?" Hmm. Well, this is a common question that everyone has encountered. First, I need to acknowledge that there's no one-size-fits-all answer here. Different people have different views on what makes sense.

I should start by saying it's up to each person's personal beliefs. Maybe explain that the meaning of life varies depending on culture, values, and individual experiences. That way, they know there's no single correct answer.

Then, I can think of some examples or sources that might give a general perspective. Like how some religions or philosophies suggest different paths. Also, mentioning scientific perspectives could be useful, like the search for answers in nature or science.

I should also consider the possibility that people are asking this out of curiosity or to find meaning in their lives. It's okay if they're looking for something they can't find on their own. But I need to make sure not to downplay the personal a

# Function Calling with LM Studio

This example shows how to use the model's `.act()` method to solve a computational problem using provided functions.
We define two utility functions (`add` and `is_prime`) and ask the model to determine if the sum of two numbers is prime.

In [3]:
import math
import lmstudio as lms

def add(a: int, b: int) -> int:
    """Given two numbers a and b, returns the sum of them."""
    return a + b

def is_prime(n: int) -> bool:
    """Given a number n, returns True if n is a prime number."""
    if n < 2:
        return False
    sqrt = int(math.sqrt(n))
    for i in range(2, sqrt):
        if n % i == 0:
            return False
    return True

model = lms.llm("qwen3-0.6b")
model.act(
  "Is the result of 12345 + 45668 a prime? Think step by step.",
  [add, is_prime],
  on_message=print,
)

AssistantResponse.from_dict({
  "content": [
    {
      "text": "<think>\nOkay, let me try to figure this out. The user is asking if the sum of 12345 and 45668 is a prime number. First, I need to calculate that sum. Let me do that addition.\n\nAdding them: 12,345 + 45,668. Let's break it down. Adding the thousands first: 10,000 + 40,000 is 50,000. Then 2,345 + 5,668. Hmm, 2,000 + 5,000 is 7,000, and 345 + 668 is 1,013. So total sum is 50,000 + 7,000 = 57,000 plus 1,013 equals 58,013.\n\nNow, check if 58,013 is a prime number. Prime numbers are numbers greater than 1 that have no divisors other than 1 and themselves. To check this, I need to see if there's any number between 2 and the square root of 58,013 that divides it evenly.\n\nFirst, let me find the square root of 58,013. The square root of 58,013 is approximately sqrt(58013). Let me calculate that. Well, 240 squared is 57,600. Then 241 squared is 241*241 = let's see: 240^2 + 2*240 +1 = 57,600 + 480 + 1 = 58,081. Oh! So sqrt(58,0

ActResult(rounds=1, total_time_seconds=36.64764069998637)

# Getting Model Context Length

Retrieve the maximum context length of the loaded model, which indicates how many tokens it can process at once.

In [5]:
context_length = model.get_context_length()
context_length

8086

# Creating a Structured Output Schema

Define a Pydantic model to structure the output from the language model. This enables type-safe responses.

In [6]:
from pydantic import BaseModel

# A class based schema for a book
class BookSchema(BaseModel):
    title: str
    author: str
    year: int


# Structured Response with Schema Validation

Generate a response that conforms to our BookSchema. The model will return data in a structured format that fits our defined schema.

In [8]:
result = model.respond("Tell me about The Hobbit", response_format=BookSchema)
book = result.parsed

print(book)
#           ^
# Note that `book` is correctly typed as { title: string, author: string, year: number }


{'author': 'J.R.R. Tolkien', 'title': 'The Hobbit', 'year': 1950}


# Creating a File Utility Function

Define a helper function that creates files with given content. This will be used by the language model to generate and save content to the filesystem.

In [9]:
from pathlib import Path

def create_file(name: str, content: str):
    """Create a file with the given name and content."""
    dest_path = Path(name)
    if dest_path.exists():
        return "Error: File already exists."
    try:
        dest_path.write_text(content, encoding="utf-8")
    except Exception as exc:
        return "Error: {exc!r}"
    return "File created."


# Function Calling to Generate a File

Use the language model to create a file with AI-generated content about the meaning of life, utilizing our file creation utility function.

In [ ]:
import lmstudio as lms

model = lms.llm("qwen3-0.6b")
model.act(
  "Please create a file named output.txt with your understanding of the meaning of life.",
  [create_file],
)

# output.txt will be created in the current working directory

ActResult(rounds=2, total_time_seconds=2.787418899941258)